In [10]:
# Importando as libraries

import requests
import pandas as pd
import json
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import psycopg2


In [11]:
# Criando o dicionário com os endpoints e os descritivos simplificados

mapeamento_colunas = {
    "utilidades": {
        "1207": "pib_corrente",
        "21774": "populacao",
        "21775": "pib_per_capita",
    },
    "emprego": {
        "24369": "taxa_desocupacao",
        "24370": "idade_trabalho",
        "24371": "empregados_total",
        "24372": "empregados_publico",
        "24373": "conta_propria",
        "24374": "empregadores",
        "24375": "empregados_carteira",
        "24376": "empregados_sem_carteira",
        "24377": "empregados_privado_total",
        "24378": "forca_trabalho",
        "24379": "pessoas_ocupadas",
        "24380": "pessoas_desocupadas",
        "28543": "ocupados_rendimento",
    },
    "rendimento_habitual": {
        "433":	"ipca",
        "1619": "salario_minimo",
        "24382": "rendimento_total",
        "24383": "rendimento_com_carteira",
        "24384": "rendimento_sem_carteira",
        "24385": "rendimento_setor_privado",
        "24386": "rendimento_setor_publico",
        "24387": "rendimento_conta_propria",
        "24388": "rendimento_empregadores",
        "24399": "rendimento_privado_publico",
    }
}

In [12]:
# Definição das variáveis relevantes

# Datas de início e fim para a consulta
data_inicial = "01/01/2015"
data_final = ""

# URL base da API
BASE_URL = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json&dataInicial={}&dataFinal={}"

In [13]:
# Definição das funções relevantes

def obter_dados_api(chave):
    url = BASE_URL.format(chave, data_inicial, data_final)
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df["data"] = pd.to_datetime(df["data"], dayfirst=True)
        df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
        df.rename(columns={"valor": str(chave)}, inplace=True)
        return df
    else:
        print(f"⚠️ Erro ao buscar dados para a chave {chave}")
        return None

# 🔹 Função para processar um grupo de indicadores
def processar_grupo(grupo, colunas_mapeadas):
    endpoints = list(colunas_mapeadas.keys())
    dfs = [obter_dados_api(str(chave)) for chave in endpoints]
    dfs = [df for df in dfs if df is not None]  # Remove valores None

    if dfs:
        for df in dfs:
            df.set_index("data", inplace=True)

        df_concatenado = pd.concat(dfs, axis=1).reset_index()

        # Renomear colunas usando o mapeamento
        df_concatenado.rename(columns=colunas_mapeadas, inplace=True)

        # Remover linhas onde todas as colunas de indicadores são NaN
        df_concatenado.dropna(subset=colunas_mapeadas.values(), how="all", inplace=True)

        return df_concatenado
    return None

# 🔹 Função para inserir DataFrame no PostgreSQL
def inserir_dataframe(df, tabela_nome):
    if df is None or df.empty:
        print(f"⚠️ DataFrame para {tabela_nome} está vazio. Nenhuma inserção feita.")
        return

    df.to_sql(tabela_nome, engine, if_exists='append', index=False)
    print(f"✅ Dados inseridos na tabela {tabela_nome}")

# 🔹 Função principal
def main():
    resultados = {}

    for grupo, colunas_mapeadas in mapeamento_colunas.items():
        df_grupo = processar_grupo(grupo, colunas_mapeadas)
        if df_grupo is not None:
            resultados[grupo] = df_grupo
            print(f"✅ Dados do grupo '{grupo}' processados com sucesso!")

            # Inserir no PostgreSQL
            tabela_nome = f"indicadores_{grupo}"
            inserir_dataframe(df_grupo, tabela_nome)
        else:
            print(f"⚠️ Nenhum dado válido para o grupo '{grupo}'.")

    return resultados

In [14]:
# Realizando a conexão do Python com o PostgreSQL usando o sqlalchemy

USERNAME = "postgres"
PASSWORD = "041700"
HOST = "localhost"  # Ou endereço do servidor
PORT = "5432"
DATABASE = "indicadores_teste"

# Criando a engine do SQLAlchemy
try:
    engine = create_engine(
        f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}",
        connect_args={"options": "-c client_encoding=UTF8"}
    )
    print("✅ Engine created successfully!")
except Exception as e:
    print(f"❌ Engine creation failed: {e}")



✅ Engine created successfully!


In [15]:
# Criando a base declarativa
Base = declarative_base()

# 🔹 Definição das tabelas

class IndicadorUtilidade(Base):
    __tablename__ = "indicadores_utilidades"
    id = Column(Integer)
    data = Column(Date, nullable=False, primary_key=True)
    pib_corrente = Column(Float)
    populacao = Column(Float)
    pib_per_capita = Column(Float)

class IndicadorEmprego(Base):
    __tablename__ = "indicadores_emprego"
    id = Column(Integer)
    data = Column(Date, nullable=False, primary_key=True)
    taxa_desocupacao = Column(Float)
    idade_trabalho = Column(Float)
    empregados_total = Column(Float)
    empregados_publico = Column(Float)
    conta_propria = Column(Float)
    empregadores = Column(Float)
    empregados_carteira = Column(Float)
    empregados_sem_carteira = Column(Float)
    empregados_privado_total = Column(Float)
    forca_trabalho = Column(Float)
    pessoas_ocupadas = Column(Float)
    pessoas_desocupadas = Column(Float)
    ocupados_rendimento = Column(Float)

class IndicadorRendimento(Base):
    __tablename__ = "indicadores_rendimento_habitual"
    id = Column(Integer)
    data = Column(Date, nullable=False, primary_key=True)
    ipca = Column(Float)
    salario_minimo = Column(Float)
    rendimento_total = Column(Float)
    rendimento_com_carteira = Column(Float)
    rendimento_sem_carteira = Column(Float)
    rendimento_setor_privado = Column(Float)
    rendimento_setor_publico = Column(Float)
    rendimento_conta_propria = Column(Float)
    rendimento_empregadores = Column(Float)
    rendimento_privado_publico = Column(Float)


# 🔹 Criar as tabelas no banco de dados
Base.metadata.create_all(engine)

# 🔹 Criar uma sessão para interagir com o banco
Session = sessionmaker(bind=engine)
session = Session()

print("✅ Conexão bem-sucedida e tabelas criadas!")

✅ Conexão bem-sucedida e tabelas criadas!


In [16]:
# Executar a função principal
if __name__ == "__main__":
    dataframes_por_tema = main()

    # Exemplo: Acessando um DataFrame específico
    print("\n📊 Exemplo de DataFrame para 'emprego':")
    print(dataframes_por_tema.get("emprego", "Nenhum dado encontrado."))

✅ Dados do grupo 'utilidades' processados com sucesso!
✅ Dados inseridos na tabela indicadores_utilidades
✅ Dados do grupo 'emprego' processados com sucesso!
✅ Dados inseridos na tabela indicadores_emprego
✅ Dados do grupo 'rendimento_habitual' processados com sucesso!
✅ Dados inseridos na tabela indicadores_rendimento_habitual

📊 Exemplo de DataFrame para 'emprego':
          data  taxa_desocupacao  idade_trabalho  empregados_total  \
0   2015-01-01               6.9          159848             65074   
1   2015-02-01               7.5          160018             64596   
2   2015-03-01               8.0          160189             64137   
3   2015-04-01               8.1          160354             64224   
4   2015-05-01               8.3          160525             64165   
..         ...               ...             ...               ...   
117 2024-10-01               6.2          176568             72260   
118 2024-11-01               6.1          176689             72387   
